# <u><b>TASK 1 : Feature Engineering Submission </b></u> - Arin Dhawan


<u><b>Aim</b></u>: To come up with 5 new features apart from the existing and in-development ones.

There are certain code presumptions I made to implement these features individually and a final code implementation in-part with the given MVP implementation. 

We assume there is an existing DB <i>'trades'</i> within which we implement these features.

### Feature 1 : Trade Momentum

We can calcuate the Trade momentum by summing up the percentage changes for the last 10 <i>(or n trades)</i> trades. Easier identification for:

* Short-term trends 
* Potential price movement based on recent activity 

A sample implementation would be as follows:

In [1]:
def trade_momentum(trades):
    returns = trades['Close'].pct_change().dropna()
    momentum = returns.rolling(window=10).sum()             # can change with any number n 
    trades['Trade_Momentum'] = momentum
    return trades

### Feature 2: Order Cancelation Ratio

We can calculate this feature by taking into proportion of cancelled orders to total orders. This can prove us an indication into:

* Changing sentiment for buyers/sellers
* Affect on liquidity and stability 

A sample implementation would look like: 

In [3]:
def calculate_order_cancellation_ratio(trades):
    trades['Order_Cancellation_Ratio'] = trades['Cancelled_Orders'] / trades['Total_Orders']
    return trades

### Feature 3 : Volume-Weighted Average Price (VWAP) Deviation

VWAP Deviation helps us calculated how much the current price deviates from the VWAP price. It help us determine whether trades are occuring in favorable or unfavorable price relative to the avg market value.

We can implement and calculate VWAP dev as follows:

In [4]:
def calculate_vwap_deviation(trades):
    trades['VWAP'] = (trades['Volume'] * trades['Close']).cumsum() / trades['Volume'].cumsum()
    trades['VWAP_Deviation'] = (trades['Close'] - trades['VWAP']) / trades['VWAP']
    return trades

### Feature 4 : Order Flow Imbalance

This feature helps us determine the difference b/w buy and sell order relative to total number of orders. It can help us with:

* Examine the BUY/SELL pressure in the market 
* Predict price changes 

An implementation would look as follows:

In [ ]:
def calculate_order_flow_imbalance(trades):
    trades['Order_Flow_Imbalance'] = (trades['Buy_Orders'] - trades['Sell_Orders']) / (trades['Buy_Orders'] + trades['Sell_Orders'])
    return trades

### Feature 5 : Price Reversion

This indicator helps us measure the deviation from current price to 20 period moving average. It can help us look into mean-reverse opportunities where price might go back to the avg. 

In [ ]:
def calculate_price_reversion_indicator(trades):
    trades['Price_Reversion_Indicator'] = trades['Close'] - trades['Close'].rolling(window=20).mean()
    return trades


## <b>Rough Implementation </b>

The above given features can be implemented in any given db. But, here is my approach for implementing the new features cohesively in the given RL configuration. I am first going to define a seperate class definiton, then preprocess with the already existing code. 

In [ ]:
import pandas as pd
import numpy as np
import talib as ta

# GIVEN 
class TechnicalIndicators:
    def __init__(self, data):
        self.data = data

    def add_momentum_indicators(self):
        self.data['RSI'] = ta.RSI(self.data['Close'], timeperiod=14)
        self.data['MACD'], self.data['MACD_signal'], self.data['MACD_hist'] = ta.MACD(self.data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        self.data['Stoch_k'], self.data['Stoch_d'] = ta.STOCH(self.data['High'], self.data['Low'], self.data['Close'],
                                                              fastk_period=14, slowk_period=3, slowd_period=3)

    def add_volume_indicators(self):
        self.data['OBV'] = ta.OBV(self.data['Close'], self.data['Volume'])

    def add_volatility_indicators(self):
        self.data['Upper_BB'], self.data['Middle_BB'], self.data['Lower_BB'] = ta.BBANDS(self.data['Close'], timeperiod=20)
        self.data['ATR_1'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=1)
        self.data['ATR_2'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=2)
        self.data['ATR_5'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)
        self.data['ATR_10'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=10)
        self.data['ATR_20'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=20)

    def add_trend_indicators(self):
        self.data['ADX'] = ta.ADX(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['+DI'] = ta.PLUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['-DI'] = ta.MINUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['CCI'] = ta.CCI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)

    def add_other_indicators(self):
        self.data['DLR'] = np.log(self.data['Close'] / self.data['Close'].shift(1))
        self.data['TWAP'] = self.data['Close'].expanding().mean()
        self.data['VWAP'] = (self.data['Volume'] * (self.data['High'] + self.data['Low']) / 2).cumsum() / self.data['Volume'].cumsum()

    

    def add_all_indicators(self):
        self.add_momentum_indicators()
        self.add_volume_indicators()
        self.add_volatility_indicators()
        self.add_trend_indicators()
        self.add_other_indicators()
        return self.data

# my code: 
# different class similar to the Technical Indicators 

class CustomIndicators:
    def __init__(self, data):
        self.data = data

    def add_trade_momentum(self, window=10):
        self.data['trade_momentum'] = self.data['Volume'].rolling(window=window).sum()

    def add_order_cancellation_ratio(self):
        self.data['order_cancellation_ratio'] = self.data['order_cancellations'] / self.data['total_orders']

    def add_vwap_deviation(self):
        self.data['vwap_deviation'] = (self.data['Close'] - self.data['VWAP']) / self.data['VWAP']

    def add_order_flow_imbalance(self):
        self.data['order_flow_imbalance'] = self.data['buy_volume'] - self.data['sell_volume']

    def add_price_reversion(self, window=20):
        self.data['price_reversion'] = self.data['Close'] - self.data['Close'].rolling(window=window).mean()

    def add_all_custom_indicators(self):
        self.add_trade_momentum()
        self.add_order_cancellation_ratio()
        self.add_vwap_deviation()
        self.add_order_flow_imbalance()
        self.add_price_reversion()
        return self.data

# preprocess them 

# Already given :
df = pd.read_csv('../xnas-itch-20230703.tbbo.csv')  # note: change relative path (might show error if copied)
df['price']=df['price']/1e9
df['bid_px_00']=df['bid_px_00']/1e9
df['ask_px_00']=df['ask_px_00']/1e9

df['Close'] = df['price']
df['Volume'] = df['size']
df['High'] = df[['bid_px_00', 'ask_px_00']].max(axis=1)
df['Low'] = df[['bid_px_00', 'ask_px_00']].min(axis=1)
df['Open'] = df['Close'].shift(1).fillna(df['Close'])


# Changes made to add the new features 
df['liquidity'] = df['bid_sz_00'] * df['bid_px_00'] + df['ask_sz_00'] * df['ask_px_00']

ti = TechnicalIndicators(df)
df_with_indicators = ti.add_all_indicators()
ci = CustomIndicators(df_with_indicators)
df_with_all_indicators = ci.add_all_custom_indicators()
market_features_df = df_with_indicators[35:]